In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def plot_compare(origin, noise):
    plt.figure(figsize=(10,10))
    size = int(len(origin)/10)

    for i in range(10):
        origin_img = origin[i*size:(i+1)*size]
        noise_img = noise[i*size:(i+1)*size]

        for j in range(5):   
            plt.subplot(10, 10, i + 2*j*10 + 1) # 행, 열, plot번호
            plt.imshow(origin_img[j].reshape(28, 28), cmap='gray')
            plt.axis('off')
            plt.subplot(10, 10, i + (2*j+1)*10 + 1) # 행, 열, plot번호
            plt.imshow(noise_img[j].reshape(28, 28), cmap='gray')
            plt.axis('off')

    plt.show()

In [ ]:
def combine_generator(gen1, gen2):
    for (batch1, batch2) in zip(gen1, gen2):
        yield (batch1, batch2)

In [ ]:
def img_to_array_generator(gen, steps):
    comb_list = []
    
    for i in range(steps):
        comb_list.extend(next(gen))
    
    return np.asarray(comb_list)

# Basic Denoising Autoencoder

In [ ]:
import keras

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from keras.layers import Dense, Dropout, InputLayer, Flatten, Reshape

In [ ]:
from keras.models import Sequential

In [ ]:
img_gen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
origin_train_gen = img_gen.flow_from_directory('./notMNIST/original/', target_size=(28, 28), color_mode='grayscale', class_mode=None, batch_size=100, shuffle=False)

In [ ]:
noise_train_gen = img_gen.flow_from_directory('./notMNIST/noise/', target_size=(28, 28), color_mode='grayscale', class_mode=None, batch_size=100, shuffle=False)

In [ ]:
origin_train = img_to_array_generator(origin_train_gen, 140)

In [ ]:
noise_train = img_to_array_generator(noise_train_gen, 140)

In [ ]:
plot_compare(origin_train, noise_train)

In [ ]:
model = Sequential()

In [ ]:
model.add(InputLayer((28,28,1,)))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(784, activation='sigmoid'))
model.add(Reshape((28,28,1,)))

In [ ]:
model.summary()

In [ ]:
model.compile('adam', loss='mse')

In [ ]:
model.fit_generator(combine_generator(noise_train_gen, origin_train_gen), steps_per_epoch=140, epochs=100, shuffle=False)

In [ ]:
pred_train = model.predict_generator(noise_train_gen, steps=140)

In [ ]:
pred_train *= 255

In [ ]:
plot_compare(origin_train, pred_train)

# Denoising Autoencoder with CNN

In [ ]:
from keras.layers import Conv2D, MaxPool2D, Deconv2D, UpSampling2D

In [ ]:
model2 = Sequential()

In [ ]:
model2.add(InputLayer((28,28,1)))

In [ ]:
model2.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model2.add(MaxPool2D((2,2)))
model2.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model2.add(MaxPool2D((2,2)))
model2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))

In [ ]:
model2.summary()

In [ ]:
model2.add(Flatten())
model2.summary()

In [ ]:
model2.add(Dense(1000, activation='relu'))
model2.add(Dense(500, activation='relu'))
model2.add(Dense(1000, activation='relu'))
model2.add(Dense(3136, activation='relu'))
model2.add(Reshape((7, 7, 64)))

In [ ]:
model2.add(Deconv2D(32, (3, 3), activation='relu', padding='same'))

In [ ]:
model2.add(UpSampling2D((2,2)))

In [ ]:
model2.add(Deconv2D(16, (3, 3), activation='relu', padding='same'))
model2.add(UpSampling2D((2,2)))

In [ ]:
model2.add(Deconv2D(1, (3,3), activation='sigmoid', padding='same'))

In [ ]:
model2.summary()

In [ ]:
model2.compile('adam', loss='mse')

In [ ]:
model2.fit_generator(combine_generator(noise_train_gen, origin_train_gen), steps_per_epoch=140, epochs=100, shuffle=False)

In [ ]:
pred_train = model.predict_generator(noise_train_gen, steps=140)
pred_train *= 255
plot_compare(origin_train, pred_train)